In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


## RANDOM FOREST MODEL (Berna Basar, Curiosity Team)

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance # computing feature importance
from sklearn.ensemble import RandomForestClassifier
from scipy.stats.stats import pearsonr
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from seaborn import load_dataset
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

test_dt = pd.read_csv("../Data/data_test.csv")
train_dt = pd.read_csv("../Data/data_train.csv")

In [20]:
train_dt.head()

,Unnamed: 0,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,158560,No,22.31,No,No,No,0.0,10.0,No,Female,70-74,White,No,Yes,Very good,8.0,Yes,No,No
1,101198,No,25.62,No,No,No,0.0,1.0,No,Male,18-24,White,No,Yes,Good,6.0,No,No,No
2,61086,No,23.40,No,No,No,0.0,2.0,No,Female,25-29,White,No,Yes,Excellent,8.0,No,No,No
3,81468,No,19.85,No,No,No,0.0,0.0,No,Female,25-29,Asian,No,No,Very good,8.0,No,No,No
4,284782,No,41.57,No,No,No,15.0,5.0,Yes,Female,65-69,White,Yes,Yes,Fair,6.0,Yes,No,No


In [14]:
#Recoding the Diabetic column
train_dt.replace('Yes (during pregnancy)', 'Yes', inplace=True)
train_dt.replace('No, borderline diabetes', 'No', inplace=True)
print('Train data value counts for Diabetic columns:''\n', train_dt['Diabetic'].value_counts())
print('\n')
test_dt.replace('Yes (during pregnancy)', 'Yes', inplace=True)
test_dt.replace('No, borderline diabetes', 'No', inplace=True)
print('Test data value counts for Diabetic columns:''\n', train_dt['Diabetic'].value_counts())

Train data value counts for Diabetic columns:
 No     221177
Yes     34659
Name: Diabetic, dtype: int64


Test data value counts for Diabetic columns:
 No     221177
Yes     34659
Name: Diabetic, dtype: int64


In [15]:
# Getting the list of categorical variables
s = (train_dt.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease', 'SkinCancer']


In [16]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_dt[object_cols]))
OH_cols_train

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255831,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
255832,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
255833,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
255834,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [17]:
# Apply one-hot encoder to each column with categorical data
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(test_dt[object_cols]))
OH_cols_test

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63954,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
63955,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
63956,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
63957,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [18]:
# Getting column names for train data
OH_encoder.get_feature_names_out()
OH_cols_train.columns = OH_encoder.get_feature_names_out()
print('Train data:', OH_cols_train.columns)

print('\n')
# Getting column names for test data
OH_cols_test.columns = OH_encoder.get_feature_names_out()
print('Test data:', OH_cols_test.columns)


AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names_out'

In [ ]:
# Removing categorical columns (will replace with one-hot encoding)
train_dt.drop(labels=None, axis=1, columns=object_cols, level=None, inplace=True, errors='ignore')
print(train_dt)
print('\n')
test_dt.drop(labels=None, axis=1, columns=object_cols, level=None, inplace=True, errors='ignore')
print(train_dt)

In [ ]:
# Add one-hot encoded columns to numerical features
OH_train = pd.concat([train_dt, OH_cols_train], axis=1)
OH_test = pd.concat([test_dt, OH_cols_test], axis=1)
OH_train

In [ ]:
#Defining the feature and outcome variables both for test and train data
X_train = OH_train.drop(['HeartDisease_No', 'HeartDisease_Yes'], axis=1)
Y_train = OH_train['HeartDisease_Yes']
X_test = OH_test.drop(['HeartDisease_No', 'HeartDisease_Yes'], axis=1)
Y_test = OH_test['HeartDisease_Yes']
print(X_train)
print(Y_train)

In [ ]:
# Checking counts of the outcome variable
print(Y_train.value_counts())
print(Y_test.value_counts())

## Random Forest 

In [ ]:
# Random Forest
Rf = RandomForestClassifier(n_estimators = 40, max_depth= 12, random_state=0,class_weight="balanced")
Rf.fit(X_train, Y_train)
Rf_pred = Rf.predict(X_test)
conf_mat = confusion_matrix(Y_test, Rf_pred, labels=[0, 1], normalize=None)
print(conf_mat)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(Y_test, Rf_pred)
plt.show()

In [ ]:
ax = sns.heatmap(conf_mat/np.sum(conf_mat), annot=True, 
            fmt='.2%', cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(Y_test, Rf_pred))